In [1]:
import pandas as pd
pd.options.display.max_columns = None

import requests
from bs4 import BeautifulSoup
import time

In [2]:
# url = "https://www.basketball-reference.com/leagues/NBA_2020_games-april.html"
# page = pd.read_html(url)

In [3]:
# season = 2022
# month = "october"
# base_url = f"https://www.basketball-reference.com/leagues/NBA_{season}_games.html"


In [4]:
# page = requests.get("https://www.basketball-reference.com/leagues/NBA_2014_games.html")
# print(page.status_code)
# soup = BeautifulSoup(page.content, "html.parser")

# filter_div = soup.find('div', class_='filter')
# filter_div



In [5]:
# base_url = "https://www.basketball-reference.com"
# a = [base_url + link["href"] for link in filter_div.find_all("a")]
# a

In [6]:
season = ""
base_url = "https://www.basketball-reference.com"

dfs = []
for season in range(2008, 2025):
    time.sleep(10)
    print(season)
    temp_url = f"https://www.basketball-reference.com/leagues/NBA_{season}_games.html"
    page = requests.get(temp_url)
    print(page.status_code)
    soup = BeautifulSoup(page.content, "html.parser")
    filter_div = soup.find('div', class_='filter')
    if filter_div is None:
        continue
    
    links = [base_url + link["href"] for link in filter_div.find_all("a")]
    for l in links:
        time.sleep(3)
        # print(l)
        try:
            temp_df = pd.read_html(l)[0]
            temp_df['season_end_year'] = season
            temp_df['season_url'] = temp_url
            temp_df['link'] = l
            print(temp_df.shape)
        except:
            print(f"error: {season} {l}")
        dfs.append(temp_df)

master = pd.concat(dfs)
print(master.shape)
master.head()


2008
200
(11, 14)
(223, 14)
(222, 14)
(222, 14)
(191, 14)
(235, 14)
(166, 14)
(41, 14)
(6, 14)
2009
200
(25, 14)
(226, 14)
(221, 14)
(229, 14)
(181, 14)
(230, 14)
(161, 14)
(38, 14)
(5, 14)
2010
200
(38, 14)
(214, 14)
(221, 14)
(230, 14)
(181, 14)
(235, 14)
(156, 14)
(31, 14)
(7, 14)
2011
200
(43, 14)
(216, 14)
(223, 14)
(230, 14)
(179, 14)
(228, 14)
(155, 14)
(33, 14)
(5, 14)
2012
200
(56, 14)
(258, 14)
(209, 14)
(253, 14)
(226, 14)
(60, 14)
(13, 14)
2013
200
(12, 14)
(220, 14)
(224, 14)
(226, 14)
(181, 14)
(236, 14)
(166, 14)
(41, 14)
(9, 14)
2014
200
(19, 14)
(228, 14)
(219, 14)
(228, 14)
(177, 14)
(237, 14)
(162, 14)
(45, 14)
(5, 14)
2015
200
(26, 14)
(222, 14)
(233, 14)
(232, 14)
(161, 14)
(236, 14)
(160, 14)
(36, 14)
(6, 14)
2016
200
(38, 14)
(223, 14)
(228, 14)
(230, 14)
(171, 14)
(237, 14)
(147, 14)
(36, 14)
(7, 14)
2017
200
(45, 14)
(229, 14)
(232, 14)
(223, 14)
(165, 14)
(241, 14)
(141, 14)
(29, 14)
(5, 14)
2018
200
(104, 14)
(213, 14)
(227, 14)
(216, 14)
(160, 14)
(222, 14)


,Date,Start (ET),Visitor/Neutral,PTS,Home/Neutral,PTS.1,Unnamed: 6,Unnamed: 7,Attend.,Arena,Notes,season,season_url,link
0,"Tue, Oct 30, 2007",8:00p,Portland Trail Blazers,97,San Antonio Spurs,106,Box Score,NaN,18797,AT&T Center,NaN,2008,https://www.basketball-reference.com/leagues/N...,https://www.basketball-reference.com/leagues/N...
1,"Tue, Oct 30, 2007",10:30p,Utah Jazz,117,Golden State Warriors,96,Box Score,NaN,19832,Oracle Arena,NaN,2008,https://www.basketball-reference.com/leagues/N...,https://www.basketball-reference.com/leagues/N...
2,"Tue, Oct 30, 2007",10:30p,Houston Rockets,95,Los Angeles Lakers,93,Box Score,NaN,18997,STAPLES Center,NaN,2008,https://www.basketball-reference.com/leagues/N...,https://www.basketball-reference.com/leagues/N...
3,"Wed, Oct 31, 2007",7:00p,Washington Wizards,110,Indiana Pacers,119,Box Score,OT,16212,Conseco Fieldhouse,NaN,2008,https://www.basketball-reference.com/leagues/N...,https://www.basketball-reference.com/leagues/N...
4,"Wed, Oct 31, 2007",7:00p,Milwaukee Bucks,83,Orlando Magic,102,Box Score,NaN,17519,Amway Arena,NaN,2008,https://www.basketball-reference.com/leagues/N...,https://www.basketball-reference.com/leagues/N...


In [46]:
# final cleaning steps
master['season_start_year'] = master['season_end_year'] - 1
master['date'] = pd.to_datetime(master['Date'].str.split(",", n=1).str[1])


In [48]:
print(master.shape)
master.head()

(21690, 16)


,Date,Start (ET),Visitor/Neutral,PTS,Home/Neutral,PTS.1,Unnamed: 6,Unnamed: 7,Attend.,Arena,Notes,season_url,link,date,season_end_year,season_start_year
0,"Tue, Oct 30, 2007",8:00p,Portland Trail Blazers,97,San Antonio Spurs,106,Box Score,NaN,18797,AT&T Center,NaN,https://www.basketball-reference.com/leagues/N...,https://www.basketball-reference.com/leagues/N...,2007-10-30,2008,2007
1,"Tue, Oct 30, 2007",10:30p,Utah Jazz,117,Golden State Warriors,96,Box Score,NaN,19832,Oracle Arena,NaN,https://www.basketball-reference.com/leagues/N...,https://www.basketball-reference.com/leagues/N...,2007-10-30,2008,2007
2,"Tue, Oct 30, 2007",10:30p,Houston Rockets,95,Los Angeles Lakers,93,Box Score,NaN,18997,STAPLES Center,NaN,https://www.basketball-reference.com/leagues/N...,https://www.basketball-reference.com/leagues/N...,2007-10-30,2008,2007
3,"Wed, Oct 31, 2007",7:00p,Washington Wizards,110,Indiana Pacers,119,Box Score,OT,16212,Conseco Fieldhouse,NaN,https://www.basketball-reference.com/leagues/N...,https://www.basketball-reference.com/leagues/N...,2007-10-31,2008,2007
4,"Wed, Oct 31, 2007",7:00p,Milwaukee Bucks,83,Orlando Magic,102,Box Score,NaN,17519,Amway Arena,NaN,https://www.basketball-reference.com/leagues/N...,https://www.basketball-reference.com/leagues/N...,2007-10-31,2008,2007


In [31]:
# 17744
print(master.shape)
master.head()

(21690, 15)


,Date,Start (ET),Visitor/Neutral,PTS,Home/Neutral,PTS.1,Unnamed: 6,Unnamed: 7,Attend.,Arena,Notes,season,season_url,link,date
0,"Tue, Oct 30, 2007",8:00p,Portland Trail Blazers,97,San Antonio Spurs,106,Box Score,NaN,18797,AT&T Center,NaN,2008,https://www.basketball-reference.com/leagues/N...,https://www.basketball-reference.com/leagues/N...,2007-10-30
1,"Tue, Oct 30, 2007",10:30p,Utah Jazz,117,Golden State Warriors,96,Box Score,NaN,19832,Oracle Arena,NaN,2008,https://www.basketball-reference.com/leagues/N...,https://www.basketball-reference.com/leagues/N...,2007-10-30
2,"Tue, Oct 30, 2007",10:30p,Houston Rockets,95,Los Angeles Lakers,93,Box Score,NaN,18997,STAPLES Center,NaN,2008,https://www.basketball-reference.com/leagues/N...,https://www.basketball-reference.com/leagues/N...,2007-10-30
3,"Wed, Oct 31, 2007",7:00p,Washington Wizards,110,Indiana Pacers,119,Box Score,OT,16212,Conseco Fieldhouse,NaN,2008,https://www.basketball-reference.com/leagues/N...,https://www.basketball-reference.com/leagues/N...,2007-10-31
4,"Wed, Oct 31, 2007",7:00p,Milwaukee Bucks,83,Orlando Magic,102,Box Score,NaN,17519,Amway Arena,NaN,2008,https://www.basketball-reference.com/leagues/N...,https://www.basketball-reference.com/leagues/N...,2007-10-31


In [50]:
# note that this includes playoff games
master[master['Start (ET)'] != "Playoffs"].to_csv("../data/game_times_bask_ref_2008-2024.csv", index=False)
print(time.ctime())

Sat Sep  9 22:01:45 2023


In [42]:
master.tail(15)

,Date,Start (ET),Visitor/Neutral,PTS,Home/Neutral,PTS.1,Unnamed: 6,Unnamed: 7,Attend.,Arena,Notes,season,season_url,link,date
99,"Sun, Apr 14, 2024",1:00p,Washington Wizards,NaN,Boston Celtics,NaN,NaN,NaN,NaN,TD Garden,NaN,2024,https://www.basketball-reference.com/leagues/N...,https://www.basketball-reference.com/leagues/N...,2024-04-14
100,"Sun, Apr 14, 2024",1:00p,Charlotte Hornets,NaN,Cleveland Cavaliers,NaN,NaN,NaN,NaN,Rocket Mortgage Fieldhouse,NaN,2024,https://www.basketball-reference.com/leagues/N...,https://www.basketball-reference.com/leagues/N...,2024-04-14
101,"Sun, Apr 14, 2024",1:00p,Atlanta Hawks,NaN,Indiana Pacers,NaN,NaN,NaN,NaN,Gainbridge Fieldhouse,NaN,2024,https://www.basketball-reference.com/leagues/N...,https://www.basketball-reference.com/leagues/N...,2024-04-14
102,"Sun, Apr 14, 2024",1:00p,Toronto Raptors,NaN,Miami Heat,NaN,NaN,NaN,NaN,Kaseya Center,NaN,2024,https://www.basketball-reference.com/leagues/N...,https://www.basketball-reference.com/leagues/N...,2024-04-14
103,"Sun, Apr 14, 2024",1:00p,Chicago Bulls,NaN,New York Knicks,NaN,NaN,NaN,NaN,Madison Square Garden (IV),NaN,2024,https://www.basketball-reference.com/leagues/N...,https://www.basketball-reference.com/leagues/N...,2024-04-14
104,"Sun, Apr 14, 2024",1:00p,Milwaukee Bucks,NaN,Orlando Magic,NaN,NaN,NaN,NaN,Amway Center,NaN,2024,https://www.basketball-reference.com/leagues/N...,https://www.basketball-reference.com/leagues/N...,2024-04-14
105,"Sun, Apr 14, 2024",1:00p,Brooklyn Nets,NaN,Philadelphia 76ers,NaN,NaN,NaN,NaN,Wells Fargo Center,NaN,2024,https://www.basketball-reference.com/leagues/N...,https://www.basketball-reference.com/leagues/N...,2024-04-14
106,"Sun, Apr 14, 2024",3:30p,Utah Jazz,NaN,Golden State Warriors,NaN,NaN,NaN,NaN,Chase Center,NaN,2024,https://www.basketball-reference.com/leagues/N...,https://www.basketball-reference.com/leagues/N...,2024-04-14
107,"Sun, Apr 14, 2024",3:30p,Houston Rockets,NaN,Los Angeles Clippers,NaN,NaN,NaN,NaN,Crypto.com Arena,NaN,2024,https://www.basketball-reference.com/leagues/N...,https://www.basketball-reference.com/leagues/N...,2024-04-14
108,"Sun, Apr 14, 2024",3:30p,Denver Nuggets,NaN,Memphis Grizzlies,NaN,NaN,NaN,NaN,FedEx Forum,NaN,2024,https://www.basketball-reference.com/leagues/N...,https://www.basketball-reference.com/leagues/N...,2024-04-14


In [9]:
assert False

AssertionError: 

In [32]:
master['Start (ET)'].value_counts()

Start (ET)
8:00p       4718
7:00p       4267
7:30p       3179
9:00p       1867
10:30p      1805
10:00p      1573
8:30p       1512
6:00p        689
9:30p        490
3:30p        474
1:00p        323
5:00p        179
3:00p        155
4:00p         78
12:00p        77
2:00p         76
6:30p         62
2:30p         44
12:30p        34
5:30p         34
4:30p         17
1:30p         12
Playoffs      12
7:15p          2
9:45p          2
7:45p          2
9:43p          1
9:42p          1
8:01p          1
8:15p          1
8:45p          1
9:15p          1
10:15p         1
Name: count, dtype: int64

In [39]:
master[master['Start (ET)'] == "Playoffs"]['link'].values

array(['https://www.basketball-reference.com/leagues/NBA_2008_games-april.html',
       'https://www.basketball-reference.com/leagues/NBA_2009_games-april.html',
       'https://www.basketball-reference.com/leagues/NBA_2010_games-april.html',
       'https://www.basketball-reference.com/leagues/NBA_2011_games-april.html',
       'https://www.basketball-reference.com/leagues/NBA_2012_games-april.html',
       'https://www.basketball-reference.com/leagues/NBA_2013_games-april.html',
       'https://www.basketball-reference.com/leagues/NBA_2014_games-april.html',
       'https://www.basketball-reference.com/leagues/NBA_2015_games-april.html',
       'https://www.basketball-reference.com/leagues/NBA_2016_games-april.html',
       'https://www.basketball-reference.com/leagues/NBA_2017_games-april.html',
       'https://www.basketball-reference.com/leagues/NBA_2018_games-april.html',
       'https://www.basketball-reference.com/leagues/NBA_2019_games-april.html'],
      dtype=object)